In [1]:
import os
import requests
import time
import yaml
import copy
import pandas as pd

from datetime import datetime, timedelta
from utils import KISOpenAPI

class DomesticIndexFetcher:
    def __init__(self, file_path="domesticIndex.xlsx", config_path = './config/config.yaml', token_file = 'token.json'):
        self.file_path = file_path
        self.config_path = config_path
        self.token_file = token_file
        self.access_token = None
        self.init_data = pd.DataFrame()
        self.load_config()

        # 기존 파일 로드
        if os.path.exists(self.file_path):
            self.init_data = pd.read_excel(self.file_path, dtype={'idxCode': str})
            print(f"기존 데이터 로드 완료: {self.file_path}")
        else:
            print(f"새로운 데이터를 생성합니다: {self.file_path}")
        
        self.data = copy.deepcopy(self.init_data)

    def load_config(self):
        """Load API configuration from YAML file."""
        with open(self.config_path, encoding='UTF-8') as f:
            _cfg = yaml.load(f, Loader=yaml.FullLoader)
        self.app_key = _cfg['APP_KEY']
        self.app_secret = _cfg['APP_SECRET']
        self.url_base = _cfg['URL_BASE']
        self.cano = _cfg['CANO']
        self.acnt_prdt_cd = _cfg['ACNT_PRDT_CD']
        self.hts_id = _cfg['HTS_ID']

    def get_access_token(self):
        api = KISOpenAPI(config_path=self.config_path, token_file=self.token_file)
        self.access_token = api.get_token()  # 유효한 토큰을 가져오기

    def fetch_index_data(self, index_code, start_date, num_calls):
        """ 특정 지수 데이터를 수집 """
        all_data = []
        current_date = start_date

        for _ in range(num_calls):
            url = f"{self.url_base}/uapi/domestic-stock/v1/quotations/inquire-index-daily-price"
            params = {
                "FID_COND_MRKT_DIV_CODE": "U",
                "FID_INPUT_ISCD": index_code,
                "FID_INPUT_DATE_1": current_date,
                "FID_PERIOD_DIV_CODE": "D",
            }
            headers = {
                "content-type": "application/json",
                "authorization": f"Bearer {self.access_token}",
                "appkey": self.app_key,
                "appsecret": self.app_secret,
                "tr_id": "FHPUP02120000",
                "custtype": "P",
            }
            time.sleep(0.05)
            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200 and response.json().get("output2"):
                result = response.json()["output2"]
                all_data.extend(result)
                last_date = result[-1]["stck_bsop_date"]
                current_date = (datetime.strptime(last_date, "%Y%m%d") - timedelta(days=1)).strftime("%Y%m%d")
            else:
                print(f"API 호출 실패 또는 데이터 없음: {current_date}")
                break

        df = pd.DataFrame(all_data)
        if not df.empty:
            df["idxCode"] = index_code
        return df

    def update_data(self, index_codes, start_date, num_calls):
        """ 데이터 업데이트 """
        if self.access_token is None:
            self.get_access_token()

        for index_code in index_codes:
            print(f"수집 중: 지수 {index_code}")
            if not self.init_data.empty:
                existing_dates = self.init_data[self.init_data["idxCode"] == index_code]["stck_bsop_date"]
                from_date = max(existing_dates) if not existing_dates.empty else pd.to_datetime('1930-01-01')
            else:
                from_date = pd.to_datetime('1930-01-01')
            
            # 데이터 수집
            new_data = self.fetch_index_data(index_code, start_date, num_calls)
            new_data.stck_bsop_date = pd.to_datetime(new_data.stck_bsop_date, format = '%Y%m%d')
            new_data = new_data.loc[new_data.stck_bsop_date > from_date]

            if not new_data.empty:
                self.data = pd.concat([self.data, new_data], ignore_index=True).drop_duplicates()
                print(f"{index_code} 데이터 수집 및 병합 완료")
            else:
                print(f"{index_code} 데이터 수집 실패 또는 없음")

    def save_data(self):
        """ 데이터 저장 """
        self.data.drop_duplicates(inplace=True)
        self.data.stck_bsop_date = pd.to_datetime(self.data.stck_bsop_date, format = '%Y%m%d')
        self.data.idxCode = self.data.idxCode.astype(str)
        self.data.sort_values(by=["idxCode", "stck_bsop_date"], ascending=[True, False], inplace=True)
        self.data.to_excel(self.file_path, index=False)
        print(f"데이터 저장 완료: {self.file_path}")

In [3]:
if __name__ == "__main__":
    file_path = "domesticIndex.xlsx"

    # 인스턴스 생성
    fetcher = DomesticIndexFetcher(file_path)

    # 수집할 지수 코드 리스트
    index_codes = ["0001", "0002", "0003", "0004", "0005"]
    start_date = datetime.today().strftime("%Y%m%d")  # 오늘 날짜
    #start_date = '20241105'
    num_calls = 10

    # 데이터 업데이트 및 저장
    fetcher.update_data(index_codes, start_date, num_calls)
    fetcher.save_data()

새로운 데이터를 생성합니다: domesticIndex.xlsx
토큰이 만료되었거나 존재하지 않습니다. 새로 발급합니다.
새로운 토큰 발급 완료.
수집 중: 지수 0001
0001 데이터 수집 및 병합 완료
수집 중: 지수 0002
0002 데이터 수집 및 병합 완료
수집 중: 지수 0003
0003 데이터 수집 및 병합 완료
수집 중: 지수 0004
0004 데이터 수집 및 병합 완료
수집 중: 지수 0005
0005 데이터 수집 및 병합 완료
데이터 저장 완료: domesticIndex.xlsx
